# Univariate Studies

In [ ]:
import sys
import os
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# Doing Run3?
ISRUN3 = True

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb #don't necessarily need for NUMU
import nue_booster 
importlib.reload(nue_booster)
import awkward
import pandas as pd

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category", "shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    #"crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP",
    "flash_pe",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    "trk_energy_proton_v", # track energy under proton hyp
    "trk_calo_energy_y_v", # track calo energy
    #"pi0_energy2_Y", # pi0 tagger variables
    "filter_pi0",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
    "weightSpline","weightTune","weightSplineTimesTune",
    # pi0 selection variables
    "pi0_radlen1","pi0_radlen2","pi0_dot1","pi0_dot2","pi0_energy1_Y","pi0_energy2_Y",
    "pi0_dedx1_fit_Y","pi0_dedx2_fit_Y","pi0_shrscore1","pi0_shrscore2","pi0_gammadot",
    "pi0_dedx1_fit_V","pi0_dedx2_fit_V","pi0_dedx1_fit_U","pi0_dedx2_fit_U",
    "pi0_mass_Y","pi0_mass_V","pi0_mass_U",
    'topological_score','trk_theta_v',
    'trk_sce_start_x_v','trk_sce_start_y_v','trk_sce_start_z_v',
    'trk_sce_end_x_v','trk_sce_end_y_v','trk_sce_end_z_v',
    'trk_len_v','pfp_generation_v','trk_distance_v',
    'trk_mcs_muon_mom_v','trk_range_muon_mom_v',
    'trk_energy_muon','trk_energy_tot','trk_energy'
]

#remove possible redundancies
variables = list(set(variables))

### Beginning of selection-specific cuts
#### Nue
#### Loose box cuts

In [ ]:
# nue preselection
PRESQ = 'nslice == 1'
PRESQ += ' and selected == 1'
PRESQ += ' and n_tracks_contained > 0'
PRESQ += ' and shr_energy_tot_cali > 0.07'

In [ ]:
# loose box cuts
LCUTQ = PRESQ
LCUTQ += ' and n_showers_contained == 1'
LCUTQ += ' and hits_ratio > 0.5'
LCUTQ += ' and tksh_distance < 6.0'
LCUTQ += ' and shr_tkfit_2cm_dedx_avg < 4.0'
LCUTQ += ' and tksh_angle > -0.9'
LCUTQ += ' and trkpid < 0.1'
LCUTQ += ' and shr_score < 0.40'
LCUTQ += ' and CosmicIP > 20.'

##### Tight box cuts

In [ ]:
# tight box cuts
TCUTQ = LCUTQ
TCUTQ += ' and tksh_distance < 4.0'
#TCUTQ += ' and (shr_tkfit_2cm_nhits_Y > 0)'
#TCUTQ += ' and (shr_tkfit_2cm_dedx_Y < 4.0 and shr_tkfit_2cm_dedx_U < 4.0 and shr_tkfit_2cm_dedx_V < 4.0)'
#TCUTQ += ' and (shr_tkfit_gap10_dedx_Y < 4.5)'
#
TCUTQ += ' and (shr_tkfit_2cm_nhits_tot > 1)'
TCUTQ += ' and (shr_tkfit_2cm_dedx_avg < 3.8)'
TCUTQ += ' and (shr_tkfit_gap10_nhits_tot > 1)'
TCUTQ += ' and (shr_tkfit_gap10_dedx_avg < 3.8)'
#
TCUTQ += ' and tksh_angle > -0.9 and tksh_angle < 0.75'
TCUTQ += ' and shrmoliereavg > 2 and shrmoliereavg < 10'
TCUTQ += ' and trkpid < 0.02'
TCUTQ += ' and n_showers_contained == 1'
TCUTQ += ' and shr_score < 0.275'
TCUTQ += ' and hits_ratio > 0.60'
TCUTQ += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=40 or secondshower_Y_vtxdist>=100)'
TCUTQ += ' and subcluster > 7'
TCUTQ += ' and trkfit < 0.70'
TCUTQ += ' and trkshrhitdist2 < 1.5'

#### pi0

In [ ]:
# pi0 selection
LOOSE = True
if (LOOSE):
    SCORECUT = 0.8 # 0.75 #75 # max track score
    DVTX = 3.0 # 3. # distance from vertex of each shower
    VTXDOT = 0.8 # dot product between each shower's direction and the vtx -> shr start vector
    EMIN1 =  50. # leading photon min energy
    EMIN2 =  20. #20. # 20. # subleading photon min energy
    GAMMADOT = 0.94 # max dot product between showres
    DEDXCUT = 0.0 # MeV/cm cut on leading shower only
else:
    SCORECUT = 0.5 # 0.75 #75 # max track score
    DVTX = 3.0 # 3. # distance from vertex of each shower
    VTXDOT = 0.8 # dot product between each shower's direction and the vtx -> shr start vector
    EMIN1 =  60. # leading photon min energy
    EMIN2 =  40. #20. # 20. # subleading photon min energy
    GAMMADOT = 0.94 # max dot product between showres
    DEDXCUT = 1.0 # MeV/cm cut on leading shower only

PI0SEL = 'nslice == 1'
PI0SEL += ' & pi0_shrscore1 < %f & pi0_shrscore2 < %f'%(SCORECUT,SCORECUT)
PI0SEL += '& pi0_dot1  > %f & pi0_dot2 > %f '%(VTXDOT,VTXDOT)
PI0SEL += ' & pi0_radlen1 > %f & pi0_radlen2 > %f & pi0_gammadot < %f '%(DVTX,DVTX,GAMMADOT)
PI0SEL += ' & pi0_energy1_Y > %f & pi0_energy2_Y > %f'%(EMIN1,EMIN2)
#PI0SEL += ' and (filter_pi0 == 1 or bnbdata==1 or extdata==1)'
PI0SEL += ' and pi0_dedx1_fit_Y >= %f'%DEDXCUT

#### NUMU Constraint

In [ ]:
####################################
# Cuts for NUMU constraint selection
####################################

#updated for SCE
#will swap numerical vals at end
FVx = [10,246]#[5,251]
FVy = [-110,110]
FVz = [20,986]

QUERY = '' #cuts on event-level variables
for a,APPEND in enumerate(['CV','VAR']):
    if a > 0:
        QUERY += ' and '
    QUERY += 'nslice_{} == 1'.format(APPEND)
    QUERY += ' and topological_score_{} > 0.06'.format(APPEND)
    QUERY += ' and reco_nu_vtx_sce_x_{} > FVx[0] and reco_nu_vtx_sce_x_{} < FVx[1]'.format(APPEND,APPEND)
    QUERY += ' and reco_nu_vtx_sce_y_{} > FVy[0] and reco_nu_vtx_sce_y_{} < FVy[1]'.format(APPEND,APPEND)
    QUERY += ' and reco_nu_vtx_sce_z_{} > FVz[0] and reco_nu_vtx_sce_z_{} < FVz[1]'.format(APPEND,APPEND)
    QUERY += ' and ( (reco_nu_vtx_sce_z_{} < 675) or (reco_nu_vtx_sce_z_{} > 775) )'.format(APPEND,APPEND) #avoid dead wire region    
    #if ISRUN3: QUERY += ' and (crtveto_{}!=1 or crthitpe_{} < 100.) and (_closestNuCosmicDist_{} > 20.)'.format(APPEND,APPEND,APPEND)
    
QUERY = QUERY.replace('FVx[0]',str(FVx[0]))
QUERY = QUERY.replace('FVy[0]',str(FVy[0]))
QUERY = QUERY.replace('FVz[0]',str(FVz[0]))
QUERY = QUERY.replace('FVx[1]',str(FVx[1]))
QUERY = QUERY.replace('FVy[1]',str(FVy[1]))
QUERY = QUERY.replace('FVz[1]',str(FVz[1])) 

print ("QUERY: \n",QUERY)

track_cuts = [
    ('trk_sce_start_x_v', '>', FVx[0]),
    ('trk_sce_start_x_v', '<', FVx[1]),
    ('trk_sce_start_y_v', '>', FVy[0]),
    ('trk_sce_start_y_v', '<', FVy[1]),
    ('trk_sce_start_z_v', '>', FVz[0]),
    ('trk_sce_start_z_v', '<', FVz[1]),
    ('trk_sce_end_x_v', '>', FVx[0]),
    ('trk_sce_end_x_v', '<', FVx[1]),
    ('trk_sce_end_y_v', '>', FVy[0]),
    ('trk_sce_end_y_v', '<', FVy[1]),
    ('trk_sce_end_z_v', '>', FVz[0]),
    ('trk_sce_end_z_v', '<', FVz[1]),
    ('trk_p_quality_v', '>', -0.5),
    ('trk_p_quality_v', '<', 0.5),
    ('trk_llr_pid_score_v', '>', 0.2),
    ('trk_score_v', '>', 0.8),
    ('trk_len_v', '>', 10),
    ('pfp_generation_v', '==', 2),
    ('trk_distance_v', '<', 4)
]

print("\ntrack cuts:")

cut_string = ""
for c,cut in enumerate(track_cuts):
    if c > 0: cut_string += " and "
    if type(cut[1]) == list: cut_string += "( ({} {} {}) or ({} {} {}) )".format(cut[0],cut[1][0],cut[2][0],cut[0],cut[1][1],cut[2][1])
    else: cut_string += "{} {} {}".format(cut[0],cut[1],cut[2])
print(cut_string)

## Load Dataframes and make a bunch of calculated columns.
### Final dataframe will have odd structure
#### cuts will need to look like "varname_VAR" or "varname_CV" to cut on variation or central-value example of the cuts

In [ ]:
importlib.reload(ls)
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

'''
#NUECV = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_det_var_v08_00_00_26_CV_run1_reco2_reco2.root"
NUECV = "/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0218/run1/prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2.root"
NUELY = "prodgenie_bnb_intrinsic_nue_uboone_overlay_mcc9_reco2_LYdown_v08_00_00_32_run1_reco2_reco2.root"
NUEWA = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodangle_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUEWX = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodx_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUEWYZ = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodyz_mcc9.1_v08_00_00_31_run1_reco2_reco2.root"
    
DETVAR_N_V = ['LY','WireMod angle','WireMod X','WireMod YZ']

DETVAR_S_V = [NUELY,NUEWA,NUEWX,NUEWYZ]
'''

#'''
NUMUCV = "prodgenie_bnb_nu_overlay_DetVar_CV_reco2_v08_00_00_38_run3b_reco2_reco2.root"
#NUMUCV = "/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0218/run1/prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root"
NUMULYATT = "prodgenie_bnb_nu_overlay_DetVar_LYAttenuation_v08_00_00_38_run3b_reco2_reco2.root"
NUMULYDOWN = "prodgenie_bnb_nu_overlay_DetVar_LYDown_v08_00_00_37_v2_run3b_reco2_reco2.root"
NUMURY = "prodgenie_bnb_nu_overlay_DetVar_LYRayleigh_v08_00_00_37_run3b_reco2_reco2.root"
NUMUSCE = "prodgenie_bnb_nu_overlay_DetVar_SCE_reco2_v08_00_00_38_run3b_reco2_reco2.root"
NUMUWX = "prodgenie_bnb_nu_overlay_DetVar_wiremod_ScaleX_v08_00_00_38_run3b_reco2_reco2.root"
NUMUWYZ = "prodgenie_bnb_nu_overlay_DetVar_wiremod_ScaleYZ_v08_00_00_38_run3b_reco2_reco2.root"
NUMUWAXZ = "prodgenie_bnb_nu_overlay_DetVar_WireModAngleXZ_v08_00_00_38_exe_run3b_reco2_reco2.root"
NUMUWAYZ = "prodgenie_bnb_nu_overlay_DetVar_WireModAngleYZ_v08_00_00_38_exe_run3b_reco2_reco2.root"
NUMUWDEDX = "prodgenie_bnb_nu_overlay_DetVar_wiremod_ScaledEdX_v08_00_00_39_run3b_reco2_reco2.root"
NUMURECOMB = "prodgenie_bnb_nu_overlay_DetVar_Recomb2_reco2_v08_00_00_39_run3b_reco2_reco2.root"
    
DETVAR_N_V = ['LY Attenuation','LY Down','Rayleigh','SCE',
              'WireMod X','WireMod YZ',
              'WireMod Angle XZ','WireMod Angle YZ',
             'WireMod dEdX', 'Recomb']

DETVAR_S_V = [NUMULYATT,NUMULYDOWN, NUMURY, NUMUSCE,
              NUMUWX, NUMUWYZ,
              NUMUWAXZ, NUMUWAYZ,
             NUMUWDEDX,NUMURECOMB]

#'''

#CV = uproot.open(NUECV)[fold][tree]
CV = uproot.open(ls.ntuple_path+NUMUCV)[fold][tree]

CVDF  = CV.pandas.df(variables, flatten=False)

CVDF['identifier'] = CVDF['run']*100000 + CVDF['evt']

CVDF.loc[ CVDF['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
CVDF.loc[ CVDF['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
CVDF.loc[ CVDF['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
CVDF.loc[ np.isnan(CVDF['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

#'''

M_mu = 0.105 #GeV/c
M_p = 0.938 #GeV/c


trk_llr_pid_v = CV.array('trk_llr_pid_score_v')
trk_energy_proton_v = CV.array('trk_energy_proton_v')
trk_calo_energy_y_v = CV.array('trk_calo_energy_y_v')
trk_id = CV.array('trk_id')-1 # I think we need this -1 to get the right result
trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
CVDF['trkpid'] = trk_llr_pid_v_sel
CVDF['protonenergy'] = trk_energy_proton_sel
CVDF['trackcaloenergy'] = trk_calo_energy_y_sel
CVDF['subcluster'] = CVDF['shrsubclusters0'] + CVDF['shrsubclusters1'] + CVDF['shrsubclusters2']
CVDF['trkfit'] = CVDF['shr_tkfit_npointsvalid'] / CVDF['shr_tkfit_npoints']
CVDF['anglediff_Y'] = np.abs(CVDF['secondshower_Y_dir']-CVDF['shrclusdir2'])

CVDF['shr_tkfit_2cm_nhits_tot'] = (CVDF['shr_tkfit_2cm_nhits_Y']+CVDF['shr_tkfit_2cm_nhits_U']+CVDF['shr_tkfit_2cm_nhits_V'])
CVDF['shr_tkfit_2cm_dedx_avg'] = (CVDF['shr_tkfit_2cm_nhits_Y']*CVDF['shr_tkfit_2cm_dedx_Y'] + CVDF['shr_tkfit_2cm_nhits_U']*CVDF['shr_tkfit_2cm_dedx_U'] + CVDF['shr_tkfit_2cm_nhits_V']*CVDF['shr_tkfit_2cm_dedx_V'])/CVDF['shr_tkfit_2cm_nhits_tot']
CVDF['shr_tkfit_gap10_nhits_tot'] = (CVDF['shr_tkfit_gap10_nhits_Y']+CVDF['shr_tkfit_gap10_nhits_U']+CVDF['shr_tkfit_gap10_nhits_V'])
CVDF['shr_tkfit_gap10_dedx_avg'] = (CVDF['shr_tkfit_gap10_nhits_Y']*CVDF['shr_tkfit_gap10_dedx_Y'] + CVDF['shr_tkfit_gap10_nhits_U']*CVDF['shr_tkfit_gap10_dedx_U'] + CVDF['shr_tkfit_gap10_nhits_V']*CVDF['shr_tkfit_gap10_dedx_V'])/CVDF['shr_tkfit_gap10_nhits_tot']

#cuts used for muon selection
CVDF['trk_p_quality_v'] = (CVDF['trk_mcs_muon_mom_v']-CVDF['trk_range_muon_mom_v'])/CVDF['trk_range_muon_mom_v']
CVDF['trk_costheta_v'] = CVDF['trk_theta_v'].apply(lambda x: np.cos(x))
CVDF['reco_nu_e_range'] = CVDF["trk_energy_muon"] + (CVDF["trk_energy_tot"] - CVDF["trk_energy"]) + M_mu
CVDF['reco_muon_e_range'] = CVDF["trk_energy_muon"] + M_mu

CVDF["is_signal"] = CVDF["category"] == 11

INTERCEPT = 0.0
SLOPE = 0.83

CVDF["reco_e"] = (CVDF["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + CVDF["trk_energy_tot"] #for showers

#'''

#QUERY = "nslice == 1"


#CVDF = CVDF.query(QUERY)

print ('there are %i CV events'%(CVDF.shape[0]))

DETSYS_SAMPLE_V = []

for i,N in enumerate(DETVAR_N_V):
    
    #if (i >= 6):
    #    continue
    
    VAR = uproot.open(ls.ntuple_path+DETVAR_S_V[i])[fold][tree]
    VARDF = VAR.pandas.df(variables, flatten=False)
    
    VARDF['identifier'] = VARDF['run']*100000 + VARDF['evt']
    
    VARDF.loc[ VARDF['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    VARDF.loc[ VARDF['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    VARDF.loc[ VARDF['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    VARDF.loc[ np.isnan(VARDF['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

    #'''
    trk_llr_pid_v = VAR.array('trk_llr_pid_score_v')
    trk_energy_proton_v = VAR.array('trk_energy_proton_v')
    trk_calo_energy_y_v = VAR.array('trk_calo_energy_y_v')
    trk_id = VAR.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    VARDF['trkpid'] = trk_llr_pid_v_sel
    VARDF['protonenergy'] = trk_energy_proton_sel
    VARDF['trackcaloenergy'] = trk_calo_energy_y_sel
    VARDF['subcluster'] = VARDF['shrsubclusters0'] + VARDF['shrsubclusters1'] + VARDF['shrsubclusters2']
    VARDF['trkfit'] = VARDF['shr_tkfit_npointsvalid'] / VARDF['shr_tkfit_npoints']
    VARDF['anglediff_Y'] = np.abs(VARDF['secondshower_Y_dir']-VARDF['shrclusdir2'])

    VARDF['shr_tkfit_2cm_nhits_tot'] = (VARDF['shr_tkfit_2cm_nhits_Y']+VARDF['shr_tkfit_2cm_nhits_U']+VARDF['shr_tkfit_2cm_nhits_V'])
    VARDF['shr_tkfit_2cm_dedx_avg'] = (VARDF['shr_tkfit_2cm_nhits_Y']*VARDF['shr_tkfit_2cm_dedx_Y'] + VARDF['shr_tkfit_2cm_nhits_U']*VARDF['shr_tkfit_2cm_dedx_U'] + VARDF['shr_tkfit_2cm_nhits_V']*VARDF['shr_tkfit_2cm_dedx_V'])/VARDF['shr_tkfit_2cm_nhits_tot']
    VARDF['shr_tkfit_gap10_nhits_tot'] = (VARDF['shr_tkfit_gap10_nhits_Y']+VARDF['shr_tkfit_gap10_nhits_U']+VARDF['shr_tkfit_gap10_nhits_V'])
    VARDF['shr_tkfit_gap10_dedx_avg'] = (VARDF['shr_tkfit_gap10_nhits_Y']*VARDF['shr_tkfit_gap10_dedx_Y'] + VARDF['shr_tkfit_gap10_nhits_U']*VARDF['shr_tkfit_gap10_dedx_U'] + VARDF['shr_tkfit_gap10_nhits_V']*VARDF['shr_tkfit_gap10_dedx_V'])/VARDF['shr_tkfit_gap10_nhits_tot']

    VARDF["reco_e"] = (VARDF["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + VARDF["trk_energy_tot"]

    VARDF['trk_p_quality_v'] = (VARDF['trk_mcs_muon_mom_v']-VARDF['trk_range_muon_mom_v'])/VARDF['trk_range_muon_mom_v']
    VARDF['trk_costheta_v'] = VARDF['trk_theta_v'].apply(lambda x: np.cos(x))
    VARDF['reco_nu_e_range'] = VARDF["trk_energy_muon"] + (VARDF["trk_energy_tot"] - VARDF["trk_energy"]) + M_mu
    VARDF['reco_muon_e_range'] = VARDF["trk_energy_muon"] + M_mu
    
    VARDF["is_signal"] = VARDF["category"] == 11 
    #'''
    
    #VARDF = VARDF.query(QUERY)
    
    INT = pd.merge(CVDF, VARDF, how='inner', on=['identifier'],suffixes=('_CV', '_VAR'))

    print ('intersection for %15s variation has %i events'%(N,INT.shape[0]))
    
    DETSYS_SAMPLE_V.append(INT)
    
    #break

# Defining some functions

In [ ]:
####################################################
#all the code necessary to make the track-level cuts
###################################################
def _apply_track_cuts(df,variable,track_cuts,mask,fix=''):
    #need to do this fancy business with the apply function to make masks
    #fix is either "CV" or "_VAR"
    for (var,op,val) in track_cuts:
        var = var + fix
        if type(op) == list:
            #this means treat two conditions in an 'or' fashion
            or_mask1 = df[var].apply(lambda x: eval("x{}{}".format(op[0],val[0])))#or condition 1
            or_mask2 = df[var].apply(lambda x: eval("x{}{}".format(op[1],val[1])))#or condition 2
            mask *= (or_mask1 + or_mask2) #just add the booleans
        else:
            mask *= df[var].apply(lambda x: eval("x{}{}".format(op,val))) #layer on each cut mask
    VARS = (df[variable]*mask).apply(lambda x: x[x != False]) #apply mask
    VARS = VARS[VARS.apply(lambda x: len(x) > 0)] #clean up empty slices
    #fix list comprehension issue for non '_v' variables
    if "_v_" not in variable:
        VARS = VARS.apply(lambda x: x[0])

    return VARS, mask

def _select_longest(df, VARS, mask,fix):
    trk_lens = (df['trk_len_v'+fix]*mask).apply(lambda x: x[x != False])#apply mask to track lengths
    trk_lens = trk_lens[trk_lens.apply(lambda x: len(x) > 0)]#clean up empty slices
    longest_mask = trk_lens.apply(lambda x: x == x[list(x).index(max(x))])#identify longest
    VARS = (VARS*longest_mask).apply(lambda x: x[x!=False])#apply mask
    VARS = VARS[VARS.apply(lambda x: len(x) > 0)]#clean up empty slices
    if len(VARS.iloc[0]) == 1:
        VARS = VARS.apply(lambda x: x[0] if len(x)>0 else -999)#expect values, not lists, for each event
    else:
        raise ValueError(
        "There are more than one longest track per slice")

    return VARS, longest_mask

def _selection(variable, sample, query="selected==1", extra_cut=None, track_cuts=None, select_longest=True, fix='',return_fix=None):
    #fix up variable and return variable
    variable = variable + fix
    if return_fix == None:
        return_variable = variable
    else:
        return_variable = variable + return_fix
        
    sel_query = query
    #check fix
    if fix != '' and fix[0] != '_':
        fix = '_' + fix
    if fix not in ['','_CV','_VAR']:
        raise ValueError("'fix' = {} is not a valid option \n Choose '_CV' or '_VAR'")
    
    if extra_cut is not None:
        sel_query += "& %s" % extra_cut

    df = sample.copy().query(sel_query)

    track_cuts_mask = df['trk_score_v{}'.format(fix)].apply(lambda x: x == x) #all-True mask
    if track_cuts is not None:
        VARS, track_cuts_mask = _apply_track_cuts(df,variable,track_cuts,track_cuts_mask,fix=fix)
    else:
        VARS = df[variable]
    #vars is now a Series object that passes all the cuts

    if "_v_" in variable and select_longest:
        VARS, longest_mask = _select_longest(df, VARS, track_cuts_mask,fix=fix)
        
    return VARS.ravel()

In [ ]:
def COVARIANCE(n_cv,n_var):
    cov = np.empty([len(n_cv), len(n_cv)])
    cov.fill(0)

    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            cov[i][j] += (n_var[i] - n_cv[i])*(n_var[j] - n_cv[j])
            #if cov[i][j] == 0:
            #    print("cov_",i,j)
            #    print(n_var[i],n_cv[i],n_var[j],n_cv[j])
            
    frac_cov = np.empty([len(n_cv), len(n_cv)])
    corr = np.empty([len(n_cv), len(n_cv)])

    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            frac_cov[i][j] =  cov[i][j] / (n_cv[i] * n_cv[j])
            corr[i][j] = cov[i][j] / np.sqrt(cov[i][i] * cov[j][j])
    return cov,frac_cov,corr

# Plotting and Analysis

In [ ]:
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm

In [ ]:
########################################
# Export sqrt(diag(frac_cov)) to .csv file
# Note SAVEPATH /syst/
########################################

#reload in case of changes
importlib.reload(ls)

#track_cuts = None



VARIABLE = "trk_costheta_v"
BINS = np.linspace(-1,1,26)

tag = '_fullsel'

SAVEPATH = ls.main_path+"detsys\\csv\\"
#if tag:
#    SAVEPATH += tag[1:]+"\\"
if not os.path.exists(SAVEPATH):
    os.makedirs(SAVEPATH)

    
#use bins to create index labels
index_labels = []
for b,bin_edge in enumerate(BINS):
    if b == len(BINS)-1: break
    index_labels.append("{}-{}".format(round(bin_edge,2),round(BINS[b+1],2)))
    
#make DataFrame that will eventually be exported
ERRS_DF = pd.DataFrame(index=index_labels)

#make sure you have the right parameters set
print("QUERY: {}".format(QUERY))
print("track_cuts: {}".format(track_cuts))
print("using tag: {}".format(tag))

#loop through all samples
for i,df_perm in enumerate(DETSYS_SAMPLE_V):
    print("starting {}...".format(DETVAR_N_V[i]))
        
    df = df_perm.copy()

    VARS_CV = _selection(VARIABLE,
                        df,QUERY,
                        track_cuts = track_cuts,
                        select_longest = True,
                        fix = '_CV')
    VARS_VAR = _selection(VARIABLE,
                        df,QUERY,
                        track_cuts = track_cuts,
                        select_longest = True,
                        fix = '_VAR')

    bc = 0.5*(BINS[1:]+BINS[:-1])
    n_cv,_ = np.histogram(VARS_CV,bins=BINS)
    n_var,_ = np.histogram(VARS_VAR,bins=BINS)
    
    cov,frac_cov,corr = COVARIANCE(n_cv,n_var)
    error = np.sqrt(np.diag(frac_cov))
    
    #layer these results onto the dataframe
    ERRS_DF[DETVAR_N_V[i]] = np.around(error, decimals=3)
    
#add in some calculated columns
samples = list(ERRS_DF.keys())
ERRS_DF['sum'] = np.sqrt((ERRS_DF[samples]**2).sum(axis=1))
samples.remove("Recomb")
ERRS_DF['sum_noRecomb'] = np.sqrt((ERRS_DF[samples]**2).sum(axis=1))
samples.append("Recomb")
samples.remove("WireMod dEdX")
ERRS_DF['sum_nodEdX'] = np.sqrt((ERRS_DF[samples]**2).sum(axis=1))

#ERRS_DF.to_csv(SAVEPATH+'{}_{}{}_{}-{}-{}.csv'.format(VARIABLE,date_time,tag,BINS[0],BINS[-1],len(BINS)))

In [ ]:
########################################
# Make many plots with one VAR sample
########################################
importlib.reload(ls)

#potentially useful values
AVx = [-1.55,254.8]
AVy = [-115.53, 117.47]
AVz = [0.1, 1036.9]
M_mu = 0.105 #GeV/c

track_cuts = [
    ('trk_sce_start_x_v', '>', FVx[0]),
    ('trk_sce_start_x_v', '<', FVx[1]),
    ('trk_sce_start_y_v', '>', FVy[0]),
    ('trk_sce_start_y_v', '<', FVy[1]),
    ('trk_sce_start_z_v', '>', FVz[0]),
    ('trk_sce_start_z_v', '<', FVz[1]),
    ('trk_sce_end_x_v', '>', FVx[0]),
    ('trk_sce_end_x_v', '<', FVx[1]),
    ('trk_sce_end_y_v', '>', FVy[0]),
    ('trk_sce_end_y_v', '<', FVy[1]),
    ('trk_sce_end_z_v', '>', FVz[0]),
    ('trk_sce_end_z_v', '<', FVz[1]),
    ('trk_p_quality_v', '>', -0.5),
    ('trk_p_quality_v', '<', 0.5),
    ('trk_llr_pid_score_v', '>', 0.2),
    ('trk_score_v', '>', 0.8),
    ('trk_len_v', '>', 10),
    ('pfp_generation_v', '==', 2),
    ('trk_distance_v', '<', 4)
]   
track_cuts = None

variables = ['reco_nu_e_range', 'reco_muon_e_range',
            'trk_llr_pid_score_v',
            'reco_nu_vtx_sce_x','reco_nu_vtx_sce_y','reco_nu_vtx_sce_z']
titles = ['Reconstructed Neutrino Energy [GeV]','Reconstructed Muon Energy [GeV]',
         'Log-Likelihood PID Score',
          "Reconstructed Vertex X [cm]","Reconstructed Vertex Y [cm]","Reconstructed Vertex Z [cm]"]
binss = [np.linspace(0.15,1.55,15),np.linspace(0.15,1.2,15),
         np.linspace(-1,1,26),
         np.linspace(FVx[0],FVx[1],26),np.linspace(FVy[0],FVy[1],26),np.linspace(FVz[0],FVz[1],26)]

variables = ['reco_nu_e_range']
titles = ['Reconstructed Range-Based Neutrino Energy [GeV]']
binss = [np.linspace(M_mu+0.01,1.55,26)]

#select variation sample
idx = DETVAR_N_V.index('SCE') #just looking at one variation 
df = DETSYS_SAMPLE_V[idx].copy()  #avoid making changed to source
tag = None

print("QUERY: {}".format(QUERY))
print("track_cuts: {}".format(track_cuts))
for VARIABLE,TITLE,BINS in  zip(variables,titles,binss):    
    print("{}\t{}\t{}".format(VARIABLE,TITLE,BINS))
    
    SAVEPATH = ls.main_path+"detsys\\plots\\"
    if tag:
        SAVEPATH += tag[1:]+'\\'
    if not os.path.exists(SAVEPATH):
        os.makedirs(SAVEPATH)

        
    ################################
    # CV-VAR histogram comparison
    fig = plt.figure(figsize=(7,5))
    
    #get queried array of the variable
    VARS_CV = _selection(VARIABLE,
                        df,QUERY,
                        track_cuts = track_cuts,
                        select_longest = True,
                        fix = '_CV')
    VARS_VAR = _selection(VARIABLE,
                        df,QUERY,
                        track_cuts = track_cuts,
                        select_longest = True,
                        fix = '_VAR')

    WEIGHTS_CV = _selection('weightSplineTimesTune',
                            df,QUERY,
                            track_cuts = track_cuts,
                            select_longest = True,
                            fix = '_CV')
    WEIGHTS_VAR = _selection('weightSplineTimesTune',
                            df,QUERY,
                            track_cuts = track_cuts,
                            select_longest = True,
                            fix = '_VAR')

    #get number of entries in each bin for each sample. and plot hists
    n_cv, bins, p = plt.hist(VARS_CV ,bins=BINS,histtype='step',lw=2,color='k',\
             label='CV',weights=WEIGHTS_CV)
    n_var, bins, p = plt.hist(VARS_VAR,bins=BINS,histtype='step',lw=2,color='r',\
             label='var : %s'%DETVAR_N_V[idx],weights=WEIGHTS_VAR)

    bc = 0.5*(bins[1:]+bins[:-1]) #bin centers

    cov,frac_cov,corr = COVARIANCE(n_cv,n_var) #calculate various matrices
    error = np.sqrt(np.diag(frac_cov)) #systematic error is this
    #incorporate errors from fractional covariance matrix
    plt.bar(bc,height=2*(error)*n_cv,bottom=n_cv-(error)*n_cv,width=bc[1]-bc[0], 
            edgecolor='gray',color='None',lw=2)
    plt.xlabel(TITLE)
    plt.ylabel('Num. Entries')
    plt.ylim(0,plt.gca().get_ylim()[1]*1.5)
    plt.legend(fontsize=16,loc=1)
    plt.show()
    plt.tight_layout()
    #fig.savefig(SAVEPATH+VARIABLE+"_"+date_time+"_{}{}.pdf".format(DETVAR_N_V[idx],tag))

    #########################################
    #Show the variance of the variable
    fig = plt.figure(figsize=(7,7))
    
    #apply _CV cuts only for equal sized arrays
    VARS_VAR_CV = _selection(VARIABLE,
                        df,QUERY,
                        track_cuts = track_cuts,
                        select_longest = True,
                        fix = '_CV',
                        return_fix = '_VAR')
    #trim the out of range bits
    mask_inrange = VARS_CV<BINS[-1]
    mask_inrange *= VARS_CV>BINS[0]
    mask_inrange *= VARS_VAR_CV<BINS[-1]
    mask_inrange *= VARS_VAR_CV>BINS[0]
    #plot variance
    plt.hist2d(VARS_CV[mask_inrange],VARS_VAR_CV[mask_inrange],bins=(BINS.size-1,BINS.size-1))#,norm=LogNorm())
    plt.xlabel('%s [CV]'%TITLE)
    plt.ylabel('%s [VAR]'%TITLE)
    plt.title('variation : %s'%DETVAR_N_V[idx],fontsize=16)
    plt.plot(BINS,BINS,'r--',lw=2)
    plt.colorbar()
    plt.tight_layout()
    #fig.savefig(SAVEPATH+VARIABLE+"_"+date_time+"_{}_variation{}.pdf".format(DETVAR_N_V[idx],tag))

    ####################################
    # Make covariance plots
    fig, axes = plt.subplots(2, 1,figsize=(10,12))
    #start with fractional covariance
    ax = axes[0]
    #plot fractional covariance
    pos = ax.imshow(frac_cov, origin='lower', cmap='viridis')#,vmin=-0.1,vmax=0.1) #be consistent when comparing multiple plots
    ax.set_title("Fractional covariance matrix : %s"%DETVAR_N_V[idx])
    ax.set_ylabel("Bin number")
    ax.set_xlabel("Bin number")
    
    # Add the text
    # Limits for the extent
    x_start = 0
    x_end = len(n_cv)#-1
    y_start = 0
    y_end = len(n_cv)#-1
    size = len(n_cv)#-1
    jump_x = (x_end - x_start) / (2.0 * size)
    jump_y = (y_end - y_start) / (2.0 * size)
    x_positions = np.linspace(start=x_start, stop=x_end, num=size, endpoint=False)
    y_positions = np.linspace(start=y_start, stop=y_end, num=size, endpoint=False)
    #print the value of error on the diagonal
    for x_index, x in enumerate(x_positions):
        #for x_index, x in enumerate(x_positions):
        ERR = frac_cov[x_index, x_index]
        #label = "{:.2f}\n{} {}".format(100.*np.sqrt(ERR),int(round(n_cv[x_index])),int(round(n_var[x_index])))
        label = "{:.2f}".format(100.*np.sqrt(ERR))
        text_x = x #+ jump_x
        text_y = x #+ jump_y
        if (np.abs(ERR) > 0.05):
            ax.text(text_x, text_y, label, color='black', ha='center', va='center',fontsize=8)
        else:
            ax.text(text_x, text_y, label, color='white', ha='center', va='center',fontsize=8)
    fig.colorbar(pos, ax=ax)
    
    #start on correlation matrix
    ax = axes[1]
    ax.set_title("Correlation matrix")
    pos = ax.imshow(corr, origin='lower')
    ax.set_ylabel("Bin number")
    ax.set_xlabel("Bin number")
    fig.colorbar(pos, ax = ax)
    plt.tight_layout()
    #fig.savefig(SAVEPATH+VARIABLE+"_"+date_time+"_{}_corrs{}.pdf".format(DETVAR_N_V[idx],tag))
    plt.show()


In [ ]:
#############################################
# Loop over all systematic variations
# ...and over several varibles
#############################################
importlib.reload(ls)

#have to edit code to just accept one of these at a time
VARIABLE = "trk_p_quality_v"
TITLE = "MCS Consistency"
BINS = np.linspace(-2,5,15)

VARIABLE = "trk_len_v"
TITLE = "Track Length, Muon Candidate [cm]"
BINS = np.linspace(0,500,20)

VARIABLE = "topological_score"
TITLE = "Topological Score"
BINS = np.linspace(0,1,25)

#do this for single var...below is multivar
variables = [VARIABLE]
titles = [TITLE]
binss = [BINS]


track_cuts = [
    ('trk_sce_start_x_v', '>', FVx[0]),
    ('trk_sce_start_x_v', '<', FVx[1]),
    ('trk_sce_start_y_v', '>', FVy[0]),
    ('trk_sce_start_y_v', '<', FVy[1]),
    ('trk_sce_start_z_v', '>', FVz[0]),
    ('trk_sce_start_z_v', '<', FVz[1]),
    ('trk_sce_end_x_v', '>', FVx[0]),
    ('trk_sce_end_x_v', '<', FVx[1]),
    ('trk_sce_end_y_v', '>', FVy[0]),
    ('trk_sce_end_y_v', '<', FVy[1]),
    ('trk_sce_end_z_v', '>', FVz[0]),
    ('trk_sce_end_z_v', '<', FVz[1]),
    ('trk_p_quality_v', '>', -0.5),
    ('trk_p_quality_v', '<', 0.5),
    ('trk_llr_pid_score_v', '>', 0.2),
    ('trk_score_v', '>', 0.8),
    ('trk_len_v', '>', 10),
    ('pfp_generation_v', '==', 2),
    ('trk_distance_v', '<', 4)
]   
track_cuts = None

# multivariate study
"""
variables = ['reco_nu_e_range', 'reco_muon_e_range',
            'trk_llr_pid_score_v',
            ]
titles = ['Reconstructed Neutrino Energy [GeV]','Reconstructed Muon Energy [GeV]',
         'Log-Likelihood PID Score',
          ]
binss = [np.linspace(0.15,1.55,15),np.linspace(0.15,1.2,15),
         np.linspace(-1,1,26),
         ]
"""

tag = '_SCEstudy' #will probably help define save path

for VARIABLE,TITLE,BINS in  zip(variables,titles,binss):
    print("\n\nDOING {} NOW!!!!!!\n".format(VARIABLE))
    
    SAVEPATH = ls.main_path+"detsys\\plots\\"
    SAVEPATH += '{}\\'.format(VARIABLE)
    if tag:
        SAVEPATH += tag[1:]+'\\'
    if not os.path.exists(SAVEPATH):
        os.makedirs(SAVEPATH)

    for i,df_perm in enumerate(DETSYS_SAMPLE_V):
        print("starting {}...".format(DETVAR_N_V[i]))
        df = df_perm.copy()
        idx = i#+6 #

        ################################
        # CV-VAR histogram comparison 
        fig = plt.figure(figsize=(7,5))
        #get queried array of the variable
        VARS_CV = _selection(VARIABLE,
                            df,QUERY,
                            track_cuts = track_cuts,
                            select_longest = True,
                            fix = '_CV')
        VARS_VAR = _selection(VARIABLE,
                            df,QUERY,
                            track_cuts = track_cuts,
                            select_longest = True,
                            fix = '_VAR')
        
        WEIGHTS_CV = _selection('weightSplineTimesTune',
                                df,QUERY,
                                track_cuts = track_cuts,
                                select_longest = True,
                                fix = '_CV')
        WEIGHTS_VAR = _selection('weightSplineTimesTune',
                                df,QUERY,
                                track_cuts = track_cuts,
                                select_longest = True,
                                fix = '_VAR')
        #get n entries per bin
        n_cv, bins, p = plt.hist(VARS_CV ,bins=BINS,histtype='step',lw=2,color='k',\
                 label='CV',weights=WEIGHTS_CV)
        n_var, bins, p = plt.hist(VARS_VAR,bins=BINS,histtype='step',lw=2,color='r',\
                 label='var : %s'%DETVAR_N_V[idx],weights=WEIGHTS_VAR)
        #calc bin centers
        bc = 0.5*(bins[1:]+bins[:-1])
        #make covariance and correlation calculations
        cov,frac_cov,corr = COVARIANCE(n_cv,n_var)
        #get and plot the systematic error with these cuts + variable
        error = np.sqrt(np.diag(frac_cov))
        plt.bar(bc,height=2*(error)*n_cv,bottom=n_cv-(error)*n_cv,width=bc[1]-bc[0] ,\
                edgecolor='gray',color='None',lw=2)
        plt.xlabel(TITLE)
        plt.ylabel('Num. Entries')
        plt.ylim(0,plt.gca().get_ylim()[1]*1.5)
        plt.legend(fontsize=16,loc=1)
        plt.show()
        #fig.savefig(SAVEPATH+VARIABLE+"_"+date_time+"_{}{}.pdf".format(DETVAR_N_V[idx],tag))

        #####################################################
        #Show the variance of the variable
        fig = plt.figure(figsize=(7,7))
        #apply _CV cuts only to get equal sized arrays
        VARS_VAR_CV = _selection(VARIABLE,
                            df,QUERY,
                            track_cuts = track_cuts,
                            select_longest = True,
                            fix = '_CV',
                            return_fix = '_VAR') #VAR events that correspond to CV events that pass all cuts
        mask_inrange = VARS_CV<BINS[-1]
        mask_inrange *= VARS_CV>BINS[0]
        mask_inrange *= VARS_VAR_CV<BINS[-1]
        mask_inrange *= VARS_VAR_CV>BINS[0]
        plt.hist2d(VARS_CV[mask_inrange],VARS_VAR_CV[mask_inrange],bins=(BINS.size-1,BINS.size-1))#,norm=LogNorm())
        plt.xlabel('%s [CV]'%TITLE)
        plt.ylabel('%s [VAR]'%TITLE)
        plt.title('variation : %s'%DETVAR_N_V[idx],fontsize=16)
        plt.tight_layout()
        plt.plot(BINS,BINS,'r--',lw=2)
        plt.colorbar()
        #fig.savefig(SAVEPATH+VARIABLE+"_"+date_time+"_{}_variation{}.pdf".format(DETVAR_N_V[idx],tag))

        ########################################
        # do correlation and frac-covariance plot
        fig, axes = plt.subplots(2, 1,figsize=(10,12))
        #start with fractional covariance
        ax = axes[0]
        pos = ax.imshow(frac_cov, origin='lower', cmap='viridis')#,vmin=-0.1,vmax=0.1)
        ax.set_title("Fractional covariance matrix : %s"%DETVAR_N_V[idx])
        ax.set_ylabel("Bin number")
        ax.set_xlabel("Bin number")
        
        # Add text for errors on diagonal of frac_covariance
        # Limits for the extent
        x_start = 0
        x_end = len(n_cv)#-1
        y_start = 0
        y_end = len(n_cv)#-1
        size = len(n_cv)#-1
        jump_x = (x_end - x_start) / (2.0 * size)
        jump_y = (y_end - y_start) / (2.0 * size)
        x_positions = np.linspace(start=x_start, stop=x_end, num=size, endpoint=False)
        y_positions = np.linspace(start=y_start, stop=y_end, num=size, endpoint=False)
        for x_index, x in enumerate(x_positions):
            #for x_index, x in enumerate(x_positions):
            ERR = frac_cov[x_index, x_index]
            #label = "{:.2f}\n{} {}".format(100.*np.sqrt(ERR),int(round(n_cv[x_index])),int(round(n_var[x_index])))
            label = "{:.2f}".format(100.*np.sqrt(ERR))
            text_x = x #+ jump_x
            text_y = x #+ jump_y
            if (np.abs(ERR) > 0.05):
                ax.text(text_x, text_y, label, color='black', ha='center', va='center',fontsize=8)
            else:
                ax.text(text_x, text_y, label, color='white', ha='center', va='center',fontsize=8)

        fig.colorbar(pos, ax=ax)
        #start on correlation matrix
        ax = axes[1]
        ax.set_title("Correlation matrix")
        pos = ax.imshow(corr, origin='lower')
        ax.set_ylabel("Bin number")
        ax.set_xlabel("Bin number")
        fig.colorbar(pos, ax = ax)
        
        #fig.savefig(SAVEPATH+VARIABLE+"_"+date_time+"_{}_corrs{}.pdf".format(DETVAR_N_V[idx],tag))
        plt.show()
